In [ ]:
l # Pastikan Anda sudah mengunggah file CSV ke Google Colab, kemudian jalankan kode berikut:

# Import Library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef
from sklearn.feature_extraction.text import TfidfVectorizer

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengatasi nilai NaN pada kolom 'title_paper' dan 'title_referenced_paper' sebelum fit_transform
# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Menyusun fitur X dan label y untuk training
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Melatih model Logistic Regression
model = LogisticRegression()
model.fit(X_train_titles, y_train)

# Menyiapkan data uji
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Menghapus entri dengan nilai NaN pada kolom title_paper atau title_referenced_paper di data test
test_data_merged = test_data_merged.dropna(subset=['title_paper', 'title_referenced_paper'])

# Melakukan prediksi pada data uji
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])
y_pred = model.predict(X_test_titles)

# Instead of directly assigning to test_data, create a new column in test_data_merged
test_data_merged['is_referenced'] = y_pred

# Now, merge this prediction back into the original test_data DataFrame
test_data = pd.merge(test_data, test_data_merged[['id', 'is_referenced']], on='id', how='left')

# Now you can safely proceed with the rest of your code
# Menampilkan hasil prediksi
print(test_data[['paper', 'referenced_paper', 'is_referenced']].head())
# Evaluasi dengan MCC (jika data train memiliki label)
mcc = matthews_corrcoef(y_train, model.predict(X_train_titles))
print("Matthews Correlation Coefficient (MCC):", mcc)

# Simpan hasil prediksi ke file CSV jika perlu
test_data.to_csv('/content/predicted_test.csv', index=True)

   paper referenced_paper  is_referenced
0  p0913            p3488            0.0
1  p2971            p4337            1.0
2  p2237            p1610            1.0
3  p2876            p3212            1.0
4  p2939            p1901            1.0
Matthews Correlation Coefficient (MCC): 0.20487259032826732


In [ ]:
# Pastikan Anda sudah mengunggah file CSV ke Google Colab, kemudian jalankan kode berikut:

# Import Library
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.feature_extraction.text import TfidfVectorizer

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengatasi nilai NaN pada kolom 'title_paper' dan 'title_referenced_paper' sebelum fit_transform
# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Menyusun fitur X dan label y untuk training
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Melatih model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_titles, y_train)

# Menyiapkan data uji
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Menghapus entri dengan nilai NaN pada kolom title_paper atau title_referenced_paper di data test
test_data_merged = test_data_merged.dropna(subset=['title_paper', 'title_referenced_paper'])

# Melakukan prediksi pada data uji
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])
y_pred = model.predict(X_test_titles)

# Menambahkan hasil prediksi ke data uji
test_data_merged['is_referenced'] = y_pred

# Menggabungkan kembali hasil prediksi ke data asli
test_data = pd.merge(test_data, test_data_merged[['id', 'is_referenced']], on='id', how='left')

# Menampilkan hasil prediksi
print(test_data[['paper', 'referenced_paper', 'is_referenced']].head())

# Evaluasi dengan MCC (jika data train memiliki label)
mcc = matthews_corrcoef(y_train, model.predict(X_train_titles))
print("Matthews Correlation Coefficient (MCC):", mcc)

# Simpan hasil prediksi ke file CSV jika perlu
test_data.to_csv('/content/predicted_test.csv', index=True)


   paper referenced_paper  is_referenced
0  p0913            p3488            0.0
1  p2971            p4337            1.0
2  p2237            p1610            1.0
3  p2876            p3212            0.0
4  p2939            p1901            1.0
Matthews Correlation Coefficient (MCC): 0.9949506203874302


In [ ]:
### Random Forest tanpa Row kosong

# Import Library
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.feature_extraction.text import TfidfVectorizer

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Simpan salinan dari test_data original sebelum dimanipulasi
original_test_data = test_data.copy()

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Menyusun fitur X dan label y untuk training
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Melatih model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_titles, y_train)

# Menyiapkan data uji
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Buat salinan ID dari dataset test yang lengkap
test_ids = test_data_merged['id'].copy()

# Mencatat indeks data dengan title yang tidak lengkap
missing_indices = test_data_merged[test_data_merged['title_paper'].isna() | test_data_merged['title_referenced_paper'].isna()].index

# Mengisi nilai NaN dengan string kosong, alih-alih menggunakan dropna
test_data_merged['title_paper'] = test_data_merged['title_paper'].fillna('')
test_data_merged['title_referenced_paper'] = test_data_merged['title_referenced_paper'].fillna('')

# Melakukan prediksi pada semua data uji (tidak lagi menggunakan dropna)
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])
y_pred = model.predict(X_test_titles)

# Menambahkan hasil prediksi ke data uji
test_data_merged['is_referenced'] = y_pred

# Update dataset asli dengan hasil prediksi
result_data = pd.merge(original_test_data, test_data_merged[['id', 'is_referenced']], on='id', how='left')

# Menampilkan hasil prediksi
print(result_data[['paper', 'referenced_paper', 'is_referenced']].head())
print(f"Total rows: {len(result_data)}")
print(f"Rows with is_referenced: {result_data['is_referenced'].count()}")
print(f"Missing values: {result_data['is_referenced'].isna().sum()}")

# Evaluasi dengan MCC (jika data train memiliki label)
mcc = matthews_corrcoef(y_train, model.predict(X_train_titles))
print("Matthews Correlation Coefficient (MCC):", mcc)

# Simpan hasil prediksi ke file CSV
result_data.to_csv('/content/predicted_test.csv', index=False)

In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 3.6 MB/s eta 0:00:00


In [ ]:
# Import Library
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.feature_extraction.text import TfidfVectorizer

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengatasi nilai NaN pada kolom 'title_paper' dan 'title_referenced_paper' sebelum fit_transform
# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Menyusun fitur X dan label y untuk training
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Melatih model XGBoost
model = XGBClassifier(n_estimators=100, random_state=42)
model.fit(X_train_titles, y_train)

# Menyiapkan data uji
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Menghapus entri dengan nilai NaN pada kolom title_paper atau title_referenced_paper di data test
test_data_merged = test_data_merged.dropna(subset=['title_paper', 'title_referenced_paper'])

# Melakukan prediksi pada data uji
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])
y_pred = model.predict(X_test_titles)

# Menambahkan hasil prediksi ke data uji
test_data_merged['is_referenced'] = y_pred

# Menggabungkan kembali hasil prediksi ke data asli
test_data = pd.merge(test_data, test_data_merged[['id', 'is_referenced']], on='id', how='left')

# Menampilkan hasil prediksi
print(test_data[['paper', 'referenced_paper', 'is_referenced']].head())

# Evaluasi dengan MCC (jika data train memiliki label)
mcc = matthews_corrcoef(y_train, model.predict(X_train_titles))
print("Matthews Correlation Coefficient (MCC):", mcc)

# Simpan hasil prediksi ke file CSV jika perlu
test_data.to_csv('/content/predicted_test.csv', index=True)


   paper referenced_paper  is_referenced
0  p0913            p3488            0.0
1  p2971            p4337            0.0
2  p2237            p1610            0.0
3  p2876            p3212            1.0
4  p2939            p1901            0.0
Matthews Correlation Coefficient (MCC): 0.3019123137588781


In [ ]:
### Model GNN
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv
from torch_geometric.data import Data
import torch.optim as optim
from torch_geometric.nn import GCNConv
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics import matthews_corrcoef

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train1.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# GNN expects a graph structure, so let's create nodes and edges based on paper relationships

# Define an edge list based on the relationship between papers (i.e., referencing each other)
paper_id_mapping = {paper_id: i for i, paper_id in enumerate(papers_metadata['paper_id'].unique())}

edges = []
for i, row in train_data_merged.iterrows():
    paper_id = paper_id_mapping.get(row['paper'])
    referenced_paper_id = paper_id_mapping.get(row['referenced_paper'])
    if paper_id is not None and referenced_paper_id is not None:
        edges.append([paper_id, referenced_paper_id])

# Convert the edge list to a torch tensor
edges_tensor = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Create a Data object for PyTorch Geometric
num_nodes = len(papers_metadata)  # Number of unique papers
features = torch.tensor(X_train_titles.toarray(), dtype=torch.float)  # TF-IDF features as node features
labels = torch.tensor(y_train.values, dtype=torch.long)  # Labels for each paper

# Create the data object for PyTorch Geometric
data = Data(x=features, edge_index=edges_tensor, y=labels)

# Define a simple GCN model for node classification
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
input_dim = features.shape[1]  # TF-IDF feature dimension
hidden_dim = 32  # Hidden dimension
output_dim = 2   # Output dimension for binary classification

model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(100):  # Train for 100 epochs
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Evaluate the model on training data
model.eval()
with torch.no_grad():
    output = model(data)
    pred = output.argmax(dim=1)  # Get the class with the highest score
    mcc = matthews_corrcoef(data.y.numpy(), pred.numpy())
    print(f"Matthews Correlation Coefficient (MCC): {mcc}")

# Now for the test data predictions
# Prepare the test data similar to how we prepared the train data
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong di test data
test_data_merged['title_paper'] = test_data_merged['title_paper'].fillna('')
test_data_merged['title_referenced_paper'] = test_data_merged['title_referenced_paper'].fillna('')

# Transform the test titles using the trained vectorizer
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])

# Create the test Data object (similar to the train Data object)
test_features = torch.tensor(X_test_titles.toarray(), dtype=torch.float)
test_data_obj = Data(x=test_features, edge_index=edges_tensor, y=None)

# Get predictions for test data
model.eval()
with torch.no_grad():
    test_output = model(test_data_obj)
    test_pred = test_output.argmax(dim=1)

# Save predictions to the test data
test_data_merged['is_referenced'] = test_pred.numpy()

# Save the test data with predictions to CSV
test_data_merged[['paper', 'referenced_paper', 'is_referenced']].to_csv('/content/predicted_test.csv', index=False)

print("Predictions saved to /content/predicted_test.csv")


  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=545106 sha256=8b0d538d96267a118427009de730d902a1158921386c2262acca4d607ffe482b
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
^C


AttributeError: partially initialized module 'torch_geometric' has no attribute 'typing' (most likely due to a circular import)

In [ ]:
# Install necessary packages if not already installed
!pip install xgboost torch-geometric scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.optim as optim

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train1.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Prepare Test Data (same process as for training data)
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong di test data
test_data_merged['title_paper'] = test_data_merged['title_paper'].fillna('')
test_data_merged['title_referenced_paper'] = test_data_merged['title_referenced_paper'].fillna('')
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])

### 1. Random Forest Model
# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_titles, y_train)

# Make predictions on test data
y_pred_rf = rf_model.predict(X_test_titles)
y_prob_rf = rf_model.predict_proba(X_test_titles)[:, 1]  # For AUC calculation

# Evaluate with MCC, AUC, F1 Score, and Accuracy
mcc_rf = matthews_corrcoef(y_train, rf_model.predict(X_train_titles))
auc_rf = roc_auc_score(y_train, rf_model.predict_proba(X_train_titles)[:, 1])
f1_rf = f1_score(y_train, rf_model.predict(X_train_titles))
accuracy_rf = accuracy_score(y_train, rf_model.predict(X_train_titles))

print(f"Random Forest Model - MCC: {mcc_rf}")
print(f"Random Forest Model - AUC: {auc_rf}")
print(f"Random Forest Model - F1 Score: {f1_rf}")
print(f"Random Forest Model - Accuracy: {accuracy_rf}")

# Save Random Forest predictions
test_data_merged['is_referenced_rf'] = y_pred_rf
test_data_merged[['paper', 'referenced_paper', 'is_referenced_rf']].to_csv('/content/predicted_rf_test.csv', index=False)

### 2. XGBoost Model
# XGBoost Model
xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train_titles, y_train)

# Make predictions on test data
y_pred_xgb = xgb_model.predict(X_test_titles)
y_prob_xgb = xgb_model.predict_proba(X_test_titles)[:, 1]  # For AUC calculation

# Evaluate with MCC, AUC, F1 Score, and Accuracy
mcc_xgb = matthews_corrcoef(y_train, xgb_model.predict(X_train_titles))
auc_xgb = roc_auc_score(y_train, xgb_model.predict_proba(X_train_titles)[:, 1])
f1_xgb = f1_score(y_train, xgb_model.predict(X_train_titles))
accuracy_xgb = accuracy_score(y_train, xgb_model.predict(X_train_titles))

print(f"XGBoost Model - MCC: {mcc_xgb}")
print(f"XGBoost Model - AUC: {auc_xgb}")
print(f"XGBoost Model - F1 Score: {f1_xgb}")
print(f"XGBoost Model - Accuracy: {accuracy_xgb}")

# Save XGBoost predictions
test_data_merged['is_referenced_xgb'] = y_pred_xgb
test_data_merged[['paper', 'referenced_paper', 'is_referenced_xgb']].to_csv('/content/predicted_xgb_test.csv', index=False)

### 3. Graph Neural Network (GNN) Model
# Convert 'paper' and 'referenced_paper' to numeric paper IDs for graph construction
paper_id_mapping = {paper_id: i for i, paper_id in enumerate(papers_metadata['paper_id'].unique())}

edges = []
for i, row in train_data_merged.iterrows():
    paper_id = paper_id_mapping.get(row['paper'])
    referenced_paper_id = paper_id_mapping.get(row['referenced_paper'])
    if paper_id is not None and referenced_paper_id is not None:
        edges.append([paper_id, referenced_paper_id])

# Convert the edge list to a torch tensor
edges_tensor = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Now create a Data object for PyTorch Geometric
num_nodes = len(papers_metadata)  # Number of unique papers
features = torch.tensor(X_train_titles.toarray(), dtype=torch.float)  # TF-IDF features as node features
labels = torch.tensor(y_train.values, dtype=torch.long)  # Labels for each paper

# Create the data object for PyTorch Geometric
data = Data(x=features, edge_index=edges_tensor, y=labels)

# Define a simple GCN model for node classification
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
input_dim = features.shape[1]  # TF-IDF feature dimension
hidden_dim = 32  # Hidden dimension
output_dim = 2   # Output dimension for binary classification

gnn_model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gnn_model.parameters(), lr=0.01)

# Train the GNN model
gnn_model.train()
for epoch in range(100):  # Train for 100 epochs
    optimizer.zero_grad()
    output = gnn_model(data)
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"GNN Epoch {epoch+1}, Loss: {loss.item()}")

# Evaluate the GNN model on training data
gnn_model.eval()
with torch.no_grad():
    output = gnn_model(data)
    pred_gnn = output.argmax(dim=1)  # Get the class with the highest score
    mcc_gnn = matthews_corrcoef(data.y.numpy(), pred_gnn.numpy())
    auc_gnn = roc_auc_score(data.y.numpy(), output[:, 1].numpy())
    f1_gnn = f1_score(data.y.numpy(), pred_gnn.numpy())
    accuracy_gnn = accuracy_score(data.y.numpy(), pred_gnn.numpy())
    print(f"GNN Model - MCC: {mcc_gnn}")
    print(f"GNN Model - AUC: {auc_gnn}")
    print(f"GNN Model - F1 Score: {f1_gnn}")
    print(f"GNN Model - Accuracy: {accuracy_gnn}")

# Now for the test data predictions with GNN
# Create the test Data object for GNN (similar to the train Data object)
test_features = torch.tensor(X_test_titles.toarray(), dtype=torch.float)
test_data_obj = Data(x=test_features, edge_index=edges_tensor, y=None)

# Get predictions for test data
gnn_model.eval()
with torch.no_grad():
    test_output = gnn_model(test_data_obj)
    test_pred = test_output.argmax(dim=1)

# Save GNN predictions
test_data_merged['is_referenced_gnn'] = test_pred.numpy()
test_data_merged[['paper', 'referenced_paper', 'is_referenced_gnn']].to_csv('/content/predicted_gnn_test.csv', index=False)

print("GNN predictions saved to /content/predicted_gnn_test.csv")


Random Forest Model - MCC: 0.9990680335507922
Random Forest Model - AUC: 0.9999993485789032
Random Forest Model - F1 Score: 0.9995340167753961
Random Forest Model - Accuracy: 0.9995340167753961
XGBoost Model - MCC: 0.7246952094864898
XGBoost Model - AUC: 0.9423078948328409
XGBoost Model - F1 Score: 0.8548131921625898
XGBoost Model - Accuracy: 0.8610205032618826
GNN Epoch 1, Loss: 0.6933016180992126
GNN Epoch 11, Loss: 0.5463607907295227
GNN Epoch 21, Loss: 0.46431979537010193
GNN Epoch 31, Loss: 0.40177521109580994
GNN Epoch 41, Loss: 0.3522067666053772
GNN Epoch 51, Loss: 0.31090638041496277
GNN Epoch 61, Loss: 0.27589499950408936
GNN Epoch 71, Loss: 0.2467886358499527
GNN Epoch 81, Loss: 0.2236015647649765
GNN Epoch 91, Loss: 0.2033093422651291
GNN Model - MCC: 0.8667334087443725
GNN Model - AUC: 0.9794234261014879
GNN Model - F1 Score: 0.9332555425904318
GNN Model - Accuracy: 0.9333643988816402
GNN predictions saved to /content/predicted_gnn_test.csv


In [ ]:
# Install necessary packages if not already installed
!pip install xgboost torch-geometric scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.optim as optim

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/sampled_train.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Menggunakan TF-IDF untuk mengubah judul menjadi fitur
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_titles = vectorizer.fit_transform(train_data_merged['title_paper'] + " " + train_data_merged['title_referenced_paper'])
y_train = train_data_merged['is_referenced']

# Prepare Test Data (same process as for training data)
test_data_merged = pd.merge(test_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
test_data_merged = pd.merge(test_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong di test data
test_data_merged['title_paper'] = test_data_merged['title_paper'].fillna('')
test_data_merged['title_referenced_paper'] = test_data_merged['title_referenced_paper'].fillna('')
X_test_titles = vectorizer.transform(test_data_merged['title_paper'] + " " + test_data_merged['title_referenced_paper'])

### 1. Random Forest Model
# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_titles, y_train)

# Make predictions on test data
y_pred_rf = rf_model.predict(X_test_titles)
y_prob_rf = rf_model.predict_proba(X_test_titles)[:, 1]  # For AUC calculation

# Evaluate with MCC, AUC, F1 Score, and Accuracy
mcc_rf = matthews_corrcoef(y_train, rf_model.predict(X_train_titles))
auc_rf = roc_auc_score(y_train, rf_model.predict_proba(X_train_titles)[:, 1])
f1_rf = f1_score(y_train, rf_model.predict(X_train_titles))
accuracy_rf = accuracy_score(y_train, rf_model.predict(X_train_titles))

print(f"Random Forest Model - MCC: {mcc_rf}")
print(f"Random Forest Model - AUC: {auc_rf}")
print(f"Random Forest Model - F1 Score: {f1_rf}")
print(f"Random Forest Model - Accuracy: {accuracy_rf}")

# Save Random Forest predictions
test_data_merged['is_referenced_rf'] = y_pred_rf
test_data_merged[['paper', 'referenced_paper', 'is_referenced_rf']].to_csv('/content/predicted_rf_test.csv', index=False)

### 2. XGBoost Model
# XGBoost Model
xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train_titles, y_train)

# Make predictions on test data
y_pred_xgb = xgb_model.predict(X_test_titles)
y_prob_xgb = xgb_model.predict_proba(X_test_titles)[:, 1]  # For AUC calculation

# Evaluate with MCC, AUC, F1 Score, and Accuracy
mcc_xgb = matthews_corrcoef(y_train, xgb_model.predict(X_train_titles))
auc_xgb = roc_auc_score(y_train, xgb_model.predict_proba(X_train_titles)[:, 1])
f1_xgb = f1_score(y_train, xgb_model.predict(X_train_titles))
accuracy_xgb = accuracy_score(y_train, xgb_model.predict(X_train_titles))

print(f"XGBoost Model - MCC: {mcc_xgb}")
print(f"XGBoost Model - AUC: {auc_xgb}")
print(f"XGBoost Model - F1 Score: {f1_xgb}")
print(f"XGBoost Model - Accuracy: {accuracy_xgb}")

# Save XGBoost predictions
test_data_merged['is_referenced_xgb'] = y_pred_xgb
test_data_merged[['paper', 'referenced_paper', 'is_referenced_xgb']].to_csv('/content/predicted_xgb_test.csv', index=False)

### 3. Graph Neural Network (GNN) Model
# Convert 'paper' and 'referenced_paper' to numeric paper IDs for graph construction
paper_id_mapping = {paper_id: i for i, paper_id in enumerate(papers_metadata['paper_id'].unique())}

edges = []
for i, row in train_data_merged.iterrows():
    paper_id = paper_id_mapping.get(row['paper'])
    referenced_paper_id = paper_id_mapping.get(row['referenced_paper'])
    if paper_id is not None and referenced_paper_id is not None:
        edges.append([paper_id, referenced_paper_id])

# Convert the edge list to a torch tensor
edges_tensor = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Now create a Data object for PyTorch Geometric
num_nodes = len(papers_metadata)  # Number of unique papers
features = torch.tensor(X_train_titles.toarray(), dtype=torch.float)  # TF-IDF features as node features
labels = torch.tensor(y_train.values, dtype=torch.long)  # Labels for each paper

# Create the data object for PyTorch Geometric
data = Data(x=features, edge_index=edges_tensor, y=labels)

# Define a simple GCN model for node classification
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
input_dim = features.shape[1]  # TF-IDF feature dimension
hidden_dim = 32  # Hidden dimension
output_dim = 2   # Output dimension for binary classification

gnn_model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gnn_model.parameters(), lr=0.01)

# Train the GNN model
gnn_model.train()
for epoch in range(100):  # Train for 100 epochs
    optimizer.zero_grad()
    output = gnn_model(data)
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"GNN Epoch {epoch+1}, Loss: {loss.item()}")

# Evaluate the GNN model on training data
gnn_model.eval()
with torch.no_grad():
    output = gnn_model(data)
    pred_gnn = output.argmax(dim=1)  # Get the class with the highest score
    mcc_gnn = matthews_corrcoef(data.y.numpy(), pred_gnn.numpy())
    auc_gnn = roc_auc_score(data.y.numpy(), output[:, 1].numpy())
    f1_gnn = f1_score(data.y.numpy(), pred_gnn.numpy())
    accuracy_gnn = accuracy_score(data.y.numpy(), pred_gnn.numpy())
    print(f"GNN Model - MCC: {mcc_gnn}")
    print(f"GNN Model - AUC: {auc_gnn}")
    print(f"GNN Model - F1 Score: {f1_gnn}")
    print(f"GNN Model - Accuracy: {accuracy_gnn}")

# Now for the test data predictions with GNN
# Create the test Data object for GNN (similar to the train Data object)
test_features = torch.tensor(X_test_titles.toarray(), dtype=torch.float)
test_data_obj = Data(x=test_features, edge_index=edges_tensor, y=None)

# Get predictions for test data
gnn_model.eval()
with torch.no_grad():
    test_output = gnn_model(test_data_obj)
    test_pred = test_output.argmax(dim=1)

# Save GNN predictions
test_data_merged['is_referenced_gnn'] = test_pred.numpy()
test_data_merged[['paper', 'referenced_paper', 'is_referenced_gnn']].to_csv('/content/predicted_gnn_test.csv', index=False)

print("GNN predictions saved to /content/predicted_gnn_test.csv")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.0 MB/s eta 0:00:00
Random Forest Model - MCC: 0.9768849705341753
Random Forest Model - AUC: 0.9999525543287588
Random Forest Model - F1 Score: 0.9769650914272144
Random Forest Model - Accuracy: 0.9994757479374464
XGBoost Model - MCC: 0.227826053183956
XGBoost Model - AUC: 0.9031798758743983
XGBoost Model - F1 Score: 0.10982279588711441
XGBoost Model - Accuracy: 0.9890042183374724


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets torch scikit-learn imbalanced-learn
!pip install --upgrade transformers

In [ ]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Mengunggah file yang diperlukan
train_data = pd.read_csv('/content/drive/MyDrive/balanced_train1.csv')
papers_metadata = pd.read_csv('/content/drive/MyDrive/papers_metadata.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Menggabungkan data train dan metadata untuk mendapatkan informasi lebih tentang paper
train_data_merged = pd.merge(train_data, papers_metadata[['paper_id', 'title']], left_on='paper', right_on='paper_id', how='left')
train_data_merged = pd.merge(train_data_merged, papers_metadata[['paper_id', 'title']], left_on='referenced_paper', right_on='paper_id', suffixes=('_paper', '_referenced_paper'), how='left')

# Mengisi nilai NaN dengan string kosong
train_data_merged['title_paper'] = train_data_merged['title_paper'].fillna('')
train_data_merged['title_referenced_paper'] = train_data_merged['title_referenced_paper'].fillna('')

# Gabungkan judul paper dan referensi untuk input BERT
train_data_merged['input_text'] = train_data_merged['title_paper'] + " [SEP] " + train_data_merged['title_referenced_paper']

# Tokenization for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input text, ensuring padding and truncation
def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=512)

# Convert data into Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data_merged[['input_text', 'is_referenced']])
train_dataset = train_dataset.map(tokenize_function, batched=True)

# Load BERT Model for Sequence Classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training Arguments
training_args = TrainingArguments(
    output_dir='/content/results',          # output directory
    num_train_epochs=3,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size per device during training
    per_device_eval_batch_size=8,      # Batch size per device during evaluation
    learning_rate=2e-5,                 # Learning rate
    weight_decay=0.01,                  # Strength of weight decay
    logging_dir='/content/logs',          # Directory for storing logs
    # The 'evaluation_strategy' argument is replaced with 'eval_steps'
    # to specify the number of steps between evaluations.
    # Here, it's set to evaluate every 500 steps.
    # Adjust this value according to your needs.
    eval_steps=500
)

# Define a compute_metrics function to evaluate the model
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(torch.tensor(predictions), dim=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    auc = roc_auc_score(labels, predictions)
    mcc = matthews_corrcoef(labels, predictions)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc,
        'mcc': mcc
    }

# Define a custom Trainer class with a modified training_step
class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.
        """
        model.train()
        inputs = self._prepare_inputs(inputs)

        # Calculate loss using logits and labels (this is the key change)
        with self.compute_loss_context_manager():
            loss = model(**inputs).loss

        # Rest of the training step (update gradients, etc.)
        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            loss = self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()

# Create a CustomTrainer instance
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    compute_metrics=compute_metrics
)


# Train the model
trainer.train()

# Evaluating on the test set
test_data_merged['input_text'] = test_data_merged['title_paper'] + " [SEP] " + test_data_merged['title_referenced_paper']
test_dataset = Dataset.from_pandas(test_data_merged[['input_text', 'is_referenced']])
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Get predictions and labels
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)
true_labels = torch.tensor(predictions.label_ids)

# Evaluate predictions
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
auc = roc_auc_score(true_labels, predicted_labels)
mcc = matthews_corrcoef(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AUC: {auc}")
print(f"MCC: {mcc}")

# Save predictions to CSV
test_data_merged['is_referenced'] = predicted_labels.numpy()
test_data_merged[['id', 'is_referenced']].to_csv('/content/predicted_test_bert.csv', index=False)

print("Predictions saved to /content/predicted_test_bert.csv")


Map:   0%|          | 0/8584 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
